# Reconaissance de genre musicaux

Lionel Baptiste, Ghali El Ouarzazi, Joévin Soulenq

Basé sur les travaux de Michaël Defferrard : https://github.com/mdeff/fma

## Données

* On utilise les données de la free music archive
* On cherche à avoir X = features.mfcc[1, 139]
* On cherche à avoir Y = tracks.genre_top[]

## Prédiction du genre musical (depuis indicateurs.csv)

* On utilise le meilleur classifieur (SVM)
* On prédit le genre à partir d'une nouvelle musique
* Prédiction depuis les indicateurs MFCC présents dans le fichier **indicateurs.csv**

In [5]:
import time
import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.utils import shuffle
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, LabelBinarizer, StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import utils

In [6]:
# On charge les fichiers
tracks = utils.load('fma_metadata/tracks.csv')
features = utils.load('fma_metadata/features.csv')

np.testing.assert_array_equal(features.index, tracks.index)

tracks.shape, features.shape

((106574, 52), (106574, 518))

In [7]:
small = tracks['set', 'subset'] <= 'medium' # 'small', 'medium', 'large' ou 'full'

train = tracks['set', 'split'] == 'training'
val   = tracks['set', 'split'] == 'validation'
test  = tracks['set', 'split'] == 'test'

y_train = tracks.loc[small & train, ('track', 'genre_top')]
y_test  = tracks.loc[small & test,  ('track', 'genre_top')]
X_train = features.loc[small & train, 'mfcc'] # ['mfcc', 'spectral_contrast']
X_test  = features.loc[small & test,  'mfcc'] # ['mfcc', 'spectral_contrast']

print('{} training examples, {} testing examples'.format(y_train.size, y_test.size))
print('{} features, {} classes'.format(X_train.shape[1], np.unique(y_train).size))

19922 training examples, 2573 testing examples
140 features, 16 classes


In [8]:
# Mélange l'ensemble des attributs (indispensable pour avoir un résultat correct)
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# Standardise les données en retirant la moyenne et réduit la variance "zero mean and unit variance".
scaler = StandardScaler(copy=False)
scaler.fit_transform(X_train)
scaler.transform(X_test)

# Classification par machine à vecteurs de support (SVM)
t = time.process_time()
#clf = SVC(kernel='rbf')
clf = MLPClassifier(hidden_layer_sizes=(10), max_iter=1000)  
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)
print('Précision : {:.2%}'.format(score))
print('Durée : {:.4f}'.format(time.process_time() - t))

Précision : 57.95%
Durée : 185.0115


In [9]:
# chargement des indicateurs d'une musique à prédire
indic = pd.read_csv('indicateurs.csv', index_col=[0], header=[0,1,2])

''' # Pré-raitement, utile ?
# Mélange l'ensemble des attributs (indispensable pour avoir un résultat correct)
indic = shuffle(indic, random_state=42)

# Standardise les données en retirant la moyenne et réduit la variance "zero mean and unit variance".
scaler = StandardScaler(copy=False)
scaler.fit_transform(indic)
scaler.transform(indic)
'''

# prédiction
prediction = clf.predict(indic)
print('Genre musical trouvé: {}'.format(prediction[0]))

# prédiction avec seuils de confiance
#dec = clf.decision_function(indic)
dec = clf.predict_proba(indic)

colonnes = ['Blues', 'Classical', 'Country', 'Easy Listening', 'Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Jazz', 'Old-Time / Historic', 'Pop', 'Rock', 'Soul-RnB', 'Spoken']
data = pd.DataFrame(data=dec, columns=colonnes)
data.head()

Genre musical trouvé: Folk


,Blues,Classical,Country,Easy Listening,Electronic,Experimental,Folk,Hip-Hop,Instrumental,International,Jazz,Old-Time / Historic,Pop,Rock,Soul-RnB,Spoken
0,4.376174e-55,4.614233e-51,8.278110e-59,2.832088e-117,9.551552e-78,6.959750e-62,1.0,6.953924e-84,9.587837e-81,1.483470e-50,1.178408e-63,7.581389e-195,1.479922e-16,1.280908e-39,8.777744e-61,7.798819e-66
